# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SleepDay_Processed&Results_Files
df = pd.read_csv('SleepDayCateg_Methylome_APPRDX.csv')
print(df.shape)
df.head(2)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SleepDay_Processed&Results_Files
(458, 10446)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_SleepDay,cg03441945,cg21094410,cg22926093,cg25025609,...,cg05568438,cg20674818,cg06655187,cg20841199,cg25600270,cg06475972,cg09922935,cg13758243,cg13082482,cg12397433
0,200991620021_R01C01,3001,2,1,1,1,0.908813,0.011350,0.012197,0.022372,...,0.090138,0.861772,0.038506,0.884416,0.910356,0.024303,0.058175,0.049767,0.916950,0.131669
1,200973410167_R04C01,3007,2,1,1,1,0.866976,0.022294,0.017479,0.028596,...,0.032168,0.939768,0.044680,0.905579,0.930004,0.023501,0.018804,0.024211,0.924934,0.048185


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'MDSP_SleepDay'], axis=1)
df1.head(2)

,APPRDX,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,cg12247200,...,cg05568438,cg20674818,cg06655187,cg20841199,cg25600270,cg06475972,cg09922935,cg13758243,cg13082482,cg12397433
0,1,0.908813,0.011350,0.012197,0.022372,0.018128,0.945195,0.730494,0.017911,0.921435,...,0.090138,0.861772,0.038506,0.884416,0.910356,0.024303,0.058175,0.049767,0.916950,0.131669
1,1,0.866976,0.022294,0.017479,0.028596,0.031988,0.937886,0.646004,0.075093,0.930482,...,0.032168,0.939768,0.044680,0.905579,0.930004,0.023501,0.018804,0.024211,0.924934,0.048185


## RanFor 1000 times, CpG only 

In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1000) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
dframe1.head()

,Accuracy,Std_Dev
0,0.475,0.087321
1,0.455,0.121347
2,0.495,0.065000
3,0.490,0.094340
4,0.505,0.103562


In [6]:
dframe1.to_csv ("SleepDay_CpG_Accuracy_1000RF.csv")

## RanFor 1000 times, CpG with Severity (Normal, Mild, Severe) 

In [7]:
df2 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender'], axis=1)
df2.head(2)

,APPRDX,MDSP_SleepDay,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,...,cg05568438,cg20674818,cg06655187,cg20841199,cg25600270,cg06475972,cg09922935,cg13758243,cg13082482,cg12397433
0,1,1,0.908813,0.011350,0.012197,0.022372,0.018128,0.945195,0.730494,0.017911,...,0.090138,0.861772,0.038506,0.884416,0.910356,0.024303,0.058175,0.049767,0.916950,0.131669
1,1,1,0.866976,0.022294,0.017479,0.028596,0.031988,0.937886,0.646004,0.075093,...,0.032168,0.939768,0.044680,0.905579,0.930004,0.023501,0.018804,0.024211,0.924934,0.048185


In [8]:
np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1000) :
        print(i)
        dataset = df2.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [9]:
dframe2.to_csv ("SleepDay_CpG_Severity_Accuracy_1000RF.csv")